In [175]:
# Example of optimization by bisection line search
import math
import bisect
import numpy as np

from bokeh.models import Band, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [176]:
# Initialization 
range_min, range_max = (-1,1)
initial_guess = 0.5
initial_step_size = 0.1
epsilon = 1.0e-6

# tuples of (x, f(x)) in order.
search_grid = []

In [177]:
def f(x):
    'Function over the range to minimize in one dim'
    return 1.1*math.exp(x-0.2) + 0.8*math.exp(0.8 - x) + 2.1*np.random.random_sample()

def add_f_to_grid(x):
    'memoization of the points searched'
    global search_grid
    # Binary search would be better
    found_k = [k for k in range(len(search_grid)) if abs(x-search_grid[k][0]) <= epsilon]
    if found_k == []:
        new_pt = (x, f(x))
        bisect.insort_left(search_grid, new_pt)
        return new_pt
    else:
        return search_grid[found_k[0]]
    
def init_grid(no_pts=3, shrinkage=3.0):
    global search_grid
    half_range = (range_max - range_min)/2
    grid_x = np.linspace(range_min + half_range/shrinkage, range_max - half_range/shrinkage, no_pts)
    search_grid = list(zip(grid_x, map(f, grid_x)))

In [178]:
# Create some starting points. 
init_grid(5)

In [179]:
# Create another point
for pt in np.linspace(-0.3, 0.5, 20):
    add_f_to_grid(pt)
search_grid

[(-0.6666666666666667, 4.50901752161131),
 (-0.33333333333333337, 4.523214535750286),
 (-0.3, 4.681335427635561),
 (-0.25789473684210523, 4.481196229265457),
 (-0.21578947368421053, 3.696613029749444),
 (-0.17368421052631577, 3.704303403021852),
 (-0.13157894736842105, 4.1190921264050475),
 (-0.08947368421052632, 2.814502884300033),
 (-0.04736842105263156, 2.9366640976140803),
 (-0.0052631578947368585, 3.451029858439506),
 (0.0, 3.2691774856241156),
 (0.0368421052631579, 3.5340767953663574),
 (0.07894736842105265, 3.1515650673237987),
 (0.12105263157894736, 4.57255144028973),
 (0.1631578947368421, 2.999505202549221),
 (0.20526315789473687, 4.302570036755283),
 (0.24736842105263163, 2.7943074532466508),
 (0.28947368421052627, 4.093019889026531),
 (0.33157894736842103, 3.553654027862362),
 (0.33333333333333326, 2.9082860603678045),
 (0.3736842105263158, 3.3180933039067573),
 (0.41578947368421054, 4.236860000966445),
 (0.4578947368421053, 4.607964643819162),
 (0.5, 3.5318683678427494),
 (

In [180]:
def dm_sample(the_pt):
    'From an x,y tuple build a design matrix row'
    x = the_pt[0]
    return (1, x, x*x)

def points_design_matrix():
    'The quadratic regression design matrix'
    dm = np.empty((0,3),dtype='float')
    for row in search_grid:
        dm = np.vstack((dm, np.array(dm_sample(row), dtype='float')))
    return dm

# points_design_matrix()

In [181]:
X = points_design_matrix()
y = [z[1] for z in search_grid]

parabola_fit = np.linalg.lstsq(X,y, rcond=-1)
parabola_fit

(array([ 3.66041519, -0.84952298,  1.12815141]),
 array([8.48798602]),
 3,
 array([5.04290425, 1.52499888, 0.59221449]))

In [182]:
def quadratic(pts, coeffs):
    c = coeffs[0]
    b = coeffs[1]
    a = coeffs[2]
    y = [a * x * x + b * x + c for x in pts]
    # print(y, a, b, c)
    return y

x = [z[0] for z in search_grid]
parabola_pts = (x, quadratic(x, parabola_fit[0]))
# parabola_pts

In [183]:
def plot_search_grid(parabola_pts):
    p = figure(plot_width = 600, plot_height = 600, 
           title = 'Current points',
           x_axis_label = 'x', y_axis_label = 'f(x)')
    pts = list(zip(*search_grid))
    p.circle(pts[0], pts[1],  color = 'darkred', size=6)
    # Add a parabola approx.
    p.line(parabola_pts[0], parabola_pts[1], color = 'lightblue')
    show(p)

In [184]:
def descent_direction(x, step=initial_step_size):
    pass

In [185]:
# search_grid = [(x, f(x)) for x in np.linspace(range_min, range_max, 80)]
plot_search_grid(parabola_pts)